In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from datetime import datetime
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import classification_report
device = torch.device('cuda')
root_data_dir = 'dataset'
class_name_to_label = {name: i for i, name in enumerate(os.listdir(root_data_dir))}

transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

class RetailerDataset(Dataset):
    
    def __init__(self, root_dir, transform=None):
        self.img_paths, self.labels = zip(*[(os.path.join(root_dir, class_name, img_name), 
                                             class_name_to_label[class_name])
                    for class_name in os.listdir(root_dir) 
                    for img_name in os.listdir(os.path.join(root_dir, class_name))
                     ])
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        return image, label

retailer_set = RetailerDataset(root_data_dir, transform)
training_set, validation_set = torch.utils.data.random_split(retailer_set, [int(len(retailer_set)*0.8), 
                                                                            int(len(retailer_set)*0.2)])
training_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True, num_workers=12)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False, num_workers=12)
# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

Training set has 7200 instances
Validation set has 1800 instances


In [15]:
import torch.nn as nn
import torch.nn.functional as F

# PyTorch models inherit from torch.nn.Module
class Net(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = Net(num_classes=len(class_name_to_label), dropout=0.2)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)

In [16]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    pbar = tqdm(enumerate(training_loader), total=len(training_loader))
    for i, data in pbar:
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs.to(device))

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels.to(device))
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i % 5 == 4:
            last_loss = running_loss / 5 # loss per batch
            pbar.set_description('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [17]:
# Initializing in a separate cell so we can easily add more epochs to the same run

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 10
model.to(device)
best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)
    with torch.no_grad():
        running_vloss = 0.0
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs.to(device))
            vloss = loss_fn(voutputs, vlabels.to(device))
            running_vloss += vloss.to('cpu')
            

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


  batch 110 loss: 1.3695806980133056: 100%|███| 113/113 [00:40<00:00,  2.76it/s]


LOSS train 1.3695806980133056 valid 1.3540056943893433
EPOCH 2:


  batch 110 loss: 0.9494886279106141: 100%|███| 113/113 [00:40<00:00,  2.78it/s]


LOSS train 0.9494886279106141 valid 0.96705561876297
EPOCH 3:


...

LOSS train 0.30706451237201693 valid 0.6075456142425537
EPOCH 9:


  batch 110 loss: 0.3238751232624054: 100%|███| 113/113 [00:41<00:00,  2.76it/s]


LOSS train 0.3238751232624054 valid 0.6033531427383423
EPOCH 10:


  batch 110 loss: 0.3190418303012848: 100%|███| 113/113 [00:42<00:00,  2.68it/s]


LOSS train 0.3190418303012848 valid 0.610502302646637


In [18]:
model.train(False)
model.to(device)
preds_labels = []
gt_labels = []
with torch.no_grad():
    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs.to(device)).to('cpu')
        _, preds = torch.max(voutputs, 1)
        preds_labels.extend(preds.tolist())
        gt_labels.extend(vlabels.tolist())

In [19]:
print(classification_report(preds_labels, gt_labels))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       115
           1       0.36      0.57      0.44        76
           2       0.67      0.56      0.61       135
           3       0.94      0.92      0.93       119
           4       0.90      0.86      0.88       120
           5       0.78      0.59      0.67       177
           6       0.97      0.87      0.92       151
           7       0.65      0.93      0.76        90
           8       0.45      0.58      0.51        93
           9       0.86      0.83      0.84       111
          10       0.81      0.71      0.76       143
          11       0.93      0.96      0.95       114
          12       0.72      0.73      0.72       113
          13       0.92      0.99      0.95       104
          14       0.89      0.82      0.85       139

    accuracy                           0.78      1800
   macro avg       0.78      0.78      0.78      1800
weighted avg       0.80   